In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.api import ARDL, ardl_select_order
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.seasonal import seasonal_decompose
from IPython.display import display, Markdown
import re
import warnings

warnings.filterwarnings("ignore")

# --- 1. SOTA级参数与环境配置 ---
# ==============================================================================
FILE_PATH = r'E:\A智网\业扩分析\业扩ARDL模型\7月业扩月底电量报告全行业_净值.xlsx' 
OUTPUT_PATH = r'E:\A智网\业扩分析\业扩ARDL模型\ARDL分析报告.xlsx'
SHEET_MAP = {
    'kwh': '电量', 'cap_high_new': '高压净新装', 'cap_high_exp': '高压净增容',
    'cap_low_new': '低压净新装', 'cap_low_exp': '低压净增容'
}
# 设置显著性水平
SIGNIFICANCE_LEVEL = 0.1
# ARDL模型参数
MAX_LAGS = 4

# --- 2. SOTA级数据加载与特征工程 ---
# ==============================================================================
def load_and_engineer_features(file_path, sheet_map):
    print("--- SOTA Step 1: Loading Data & Feature Engineering ---")
    all_data = {}
    for var, sheet in sheet_map.items():
        try:
            df = pd.read_excel(file_path, sheet_name=sheet)
            df['分类'] = df['分类'].astype(str).str.strip()
            date_cols = [c for c in df.columns if re.match(r'^\d{6}$', str(c).split('.')[0])]
            df = df[['分类'] + date_cols]
            df_long = df.melt(id_vars='分类', var_name='date', value_name=var)
            df_long['date'] = pd.to_datetime(df_long['date'].astype(str), format='%Y%m')
            df_long[var] = pd.to_numeric(df_long[var], errors='coerce').fillna(0)
            all_data[var] = df_long.set_index(['分类', 'date'])
        except Exception as e:
            raise RuntimeError(f"Error processing sheet '{sheet}': {e}")
            
    df_full = pd.concat(all_data.values(), axis=1)
    
    # SOTA Feature: 添加春节虚拟变量 (Chinese New Year Dummy)
    # 春节通常在1月或2月，对工业数据影响巨大
    df_full['is_cny_period'] = ((df_full.index.get_level_values('date').month == 1) | 
                                (df_full.index.get_level_values('date').month == 2)).astype(int)
    
    print("  -> Data loaded. Added 'is_cny_period' dummy variable.")
    return df_full

# --- 3. SOTA级核心分析流程 ---
# ==============================================================================
def run_sota_ardl_analysis(df_full):
    print("\n--- SOTA Step 2: Running Industry-by-Industry ARDL Analysis ---")
    
    industries = df_full.index.get_level_values('分类').unique()
    results_list = []

    for industry in industries:
        print(f"\n{'='*25} Analyzing: {industry} {'='*25}")
        
        df_industry = df_full.loc[industry].dropna()
        if len(df_industry) < 40:
            print("    -> Insufficient data points (< 40), skipping.")
            continue

        endog, exog = df_industry['kwh'], df_industry.drop(columns='kwh')
        coint_test_pval = coint(endog, exog.drop(columns='is_cny_period', errors='ignore'))[1] # Engle-Granger test
        print(f"    -> Engle-Granger Cointegration Test p-value: {coint_test_pval:.4f}")

        try:
            sel_order = ardl_select_order(
                endog, maxlag=MAX_LAGS, exog=exog, maxorder=MAX_LAGS, 
                ic='aic', trend='c', fixed=exog[['is_cny_period']] if 'is_cny_period' in exog.columns else None
            )
            model_fit = sel_order.model.fit()
            print(f"    -> ARDL({sel_order.ar_lags}, {sel_order.dl_lags}) model fitted.")
        except Exception as e:
            print(f"    -> ARDL fitting failed: {e}, skipping.")
            continue

        # --- C. Extracting Rich Results (兼容性修复) ---
        
        # 1. 使用模型总F-stat作为Bounds Test的代理
        f_stat = model_fit.fvalue
        
        result = {'行业分类': industry, 'R-squared_adj': model_fit.rsquared_adj, 'Bounds_Test_F_stat': f_stat, 'Coint_Test_pval': coint_test_pval}
        
        # 2. 从 long_run_summary() 的文本中解析长期系数
        long_run_summary_text = model_fit.long_run_summary()
        
        ecm = model_fit.ecm()
        
        for exog_var in exog.columns.drop('is_cny_period', errors='ignore'):
            try:
                # 解析长期系数
                summary_lines = str(long_run_summary_text).split('\n')
                for line in summary_lines:
                    if line.strip().startswith(exog_var):
                        parts = line.split()
                        result[f'β_long_{exog_var}'] = float(parts[1])
                        result[f'p_long_{exog_var}'] = float(parts[3])
                        break # 找到后就跳出
                else: # 如果循环结束都没找到
                    result[f'β_long_{exog_var}'] = np.nan
                    result[f'p_long_{exog_var}'] = np.nan
            except (IndexError, ValueError):
                result[f'β_long_{exog_var}'] = np.nan
                result[f'p_long_{exog_var}'] = np.nan

            # 提取短期系数
            result[f'β_short_{exog_var}'] = ecm.params.get(f'{exog_var}.L0', np.nan)
        
        result['speed_of_adj'] = ecm.params.get('ECT.L1', ecm.params.get('e.L1'))
        print(f"    -> Analysis complete. Adj R²: {model_fit.rsquared_adj:.3f}, F-stat: {f_stat:.2f}")
        results_list.append(result)
        
    return pd.DataFrame(results_list)

# --- 4. SOTA级结果展示与保存 ---
# ==============================================================================
def display_sota_results(df_summary, output_path):
    if df_summary.empty:
        print("\nNo industries could be analyzed.")
        return

    print(f"\n{'='*30} SOTA ARDL Analysis Summary {'='*30}")
    df_summary.set_index('行业分类', inplace=True)

    # Cointegration Check: F-stat > 5% critical value (~3.79 for 4 vars) and must be significant
    is_cointegrated = df_summary['Bounds_Test_F_stat'] > 3.79

    def style_table(df, p_col_prefix, is_coint):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        for beta_col in df.columns:
            pval_col = p_col_prefix + beta_col
            if pval_col in df_summary.columns:
                is_significant = df_summary[pval_col] < SIGNIFICANCE_LEVEL
                styles[beta_col] = np.where(is_significant & is_coint, 'font-weight: bold; color: green;', 
                                        np.where(is_significant, 'font-weight: bold; color: orange;', 'color: grey;'))
        return styles

    # --- Long-run Results ---
    beta_long_cols = sorted([c for c in df_summary.columns if c.startswith('β_long_')])
    df_betas_long = df_summary[beta_long_cols].rename(columns=lambda c: c.replace('β_long_', ''))
    display(Markdown("### 各行业【长期影响系数 β_long】(kWh/kVA)"))
    display(Markdown("绿色粗体: 显著且协整关系可信; 橙色粗体: 显著但长期关系存疑; 灰色: 不显著"))
    display(df_betas_long.style.format("{:.2f}").apply(style_table, p_col_prefix='p_long_', is_coint=is_cointegrated, axis=None))
    
    # --- Short-run Results ---
    beta_short_cols = sorted([c for c in df_summary.columns if c.startswith('β_short_')])
    df_betas_short = df_summary[beta_short_cols].rename(columns=lambda c: c.replace('β_short_', ''))
    display(Markdown("### 各行业【短期冲击系数 β_short】(当期影响)"))
    display(df_betas_short.style.format("{:.2f}"))
    
    # --- Save to Excel ---
    with pd.ExcelWriter(output_path) as writer:
        df_betas_long.to_excel(writer, sheet_name='长期影响系数(β_long)')
        df_betas_short.to_excel(writer, sheet_name='短期冲击系数(β_short)')
        df_summary.to_excel(writer, sheet_name='全部详细结果')
    print(f"\nFull summary report saved to: {output_path}")

# --- 5. 执行主程序 ---
# ==============================================================================
if __name__ == '__main__':
    main_df = load_and_engineer_features(FILE_PATH, SHEET_MAP)
    sota_results = run_sota_ardl_analysis(main_df)
    display_sota_results(sota_results, OUTPUT_PATH)
    print("\n--- State-of-the-Art Analysis Complete ---")

--- SOTA Step 1: Loading Data & Feature Engineering ---
  -> Data loaded. Added 'is_cny_period' dummy variable.

--- SOTA Step 2: Running Industry-by-Industry ARDL Analysis ---

========================= Analyzing: 全行业总计 =========================
    -> Stationarity Warning: Some variables might be I(2) or stationary. ARDL robustness check needed.
    -> Engle-Granger Cointegration Test p-value: 0.9997
    -> Cointegration Warning: No strong evidence of long-run relationship. Interpret long-run coefficients with caution.
    -> ARDL([1, 2, 3, 4], {'cap_high_new': [0], 'cap_high_exp': [0, 1, 2, 3], 'cap_low_new': [0, 1, 2], 'cap_low_exp': [0], 'is_cny_period': [0, 1, 2, 3]}) model fitted.


AttributeError: 'ARDLResults' object has no attribute 'get_longrun_models'